# Bước 3: Phân tích kết quả tìm kiếm và Tra cứu Văn bản gốc (v2 - FAISS L2)

## Mục tiêu của script
Script này dùng để:
- **Đọc file kết quả thô** từ Bước 2c (Faiss L2 trên DB đã whiten).
- Với mỗi truy vấn (query) và Top-K hàng xóm (neighbors):
  - **Tra cứu câu gốc (`text`)** và **span argument (`arguments[ARG-x]`)** từ các file JSON dataset sạch (Clean_Dataset).
- **Làm giàu (enrich)** kết quả:
  - Thêm `sentence_text` và `argument_text` cho query và neighbors
  - Giữ lại khoảng cách `total_L2_distance_whitened`
  - Giữ lại `original_pt_file` để tham khảo
- Lưu ra file JSON cuối cùng đã phân tích.


## 1) Cấu hình
- `K = 5`  
  Số lượng neighbors cần xử lý cho mỗi query.  
  **Phải trùng với K ở Bước 2c** vì file input được đặt tên theo `K`.

## 2) Thiết lập đường dẫn I/O

### Thư mục
- `db_cache_dir`: chứa các file database đã gộp (không dùng trực tiếp trong bước này).
- `output_results_dir`: chứa kết quả tìm kiếm và nơi lưu output phân tích.

### Input file (kết quả thô từ Bước 2c)
- `search_results_top{K}_v2_WHITENED_L2.json`  
  Cấu trúc thường là:
  - `query_info`: metadata truy vấn
  - `neighbors`: danh sách Top-K kết quả (mỗi cái có `match_metadata`)

### Output file (kết quả cuối cùng sau tra cứu)
- `final_analyzed_results_top{K}_v2_WHITENED_L2.json`  
  Chứa text đầy đủ cho query và neighbors.

## 3) Ánh xạ thư mục vector → thư mục dataset JSON gốc (`CLEAN_DATA_MAP`)

### Mục đích
Trong metadata, mỗi mẫu có `source_dir` kiểu như:
- `span_adaptation_vectors_train_gramvar_inner_content`
- `span_adaptation_vectors_test_parave_inner_content`

Nhưng để tra text, cần biết thư mục JSON dataset sạch tương ứng:
- GramVar Train/Test
- ParaVE Train/Test

### Cấu trúc ánh xạ
`CLEAN_DATA_MAP` map từ `source_dir` → đường dẫn tới dataset sạch, ví dụ:
- Train GramVar → `Clean_Dataset/Corpus/Split_GramVar/Train`
- Test ParaVE → `Clean_Dataset/Corpus/Split_ParaVE/Test`

Nếu `source_dir` không có trong map → trả về `"N/A"` và báo lỗi ánh xạ.

## 4) Các hàm tra cứu text (có cache)

## 4.1 Hàm `get_original_data(base_data_path, corpus_id, suffix)`
### Mục đích
- Đọc file JSON dataset gốc cho một corpus cụ thể, ví dụ:
  - `{corpus_id}_train_set.json`
  - `{corpus_id}_test_set.json`

### Dùng cache để tăng tốc
Hàm được gắn decorator:
- `@functools.lru_cache(maxsize=None)`

Ý nghĩa:
- Một file JSON của mỗi corpus chỉ cần đọc **một lần**
- Các lần tra cứu tiếp theo dùng lại dữ liệu đã load trong RAM
- Giảm rất mạnh thời gian I/O khi có nhiều query/neighbors

### Xử lý lỗi
- Nếu file không tồn tại → in cảnh báo và trả `None`
- Nếu lỗi đọc file → in lỗi và trả `None`

## 4.2 Hàm `get_text_data(source_dir, corpus_id, sentence_idx, span_id)`
### Mục đích
Tra cứu:
- `sentence_text`: câu đầy đủ (`item['text']`)
- `argument_text`: text của span argument (`item['arguments'][ARG-x]`)

### Luồng xử lý
1. Lấy `base_data_path` từ `CLEAN_DATA_MAP[source_dir]`
   - Nếu không có → trả `"N/A (Lỗi ánh xạ source_dir)"`

2. Xác định suffix file JSON:
   - Nếu `source_dir` chứa `'test'` → `_test_set.json`
   - Ngược lại → `_train_set.json`

3. Load file corpus (có cache):
   - `original_data = get_original_data(base_data_path, corpus_id, file_suffix)`
   - Nếu `None` → trả `"N/A (Không tìm thấy file data)"`

4. Truy cập đúng câu:
   - `item = original_data[sentence_idx]`
   - Lấy câu:
     - `sentence_text = item.get('text', 'N/A (Không có text)')`

5. Chuyển đổi span_id sang key trong JSON:
   - metadata có thể lưu `ARG_1`
   - nhưng trong file JSON thường là `ARG-1`
   - chuyển bằng:
     - `arg_key = span_id.replace('_', '-')`

6. Lấy argument text:
   - `argument_text = item.get('arguments', {}).get(arg_key, 'N/A (Không có arg)')`

### Xử lý lỗi
- Nếu `sentence_idx` vượt quá độ dài list → `"N/A (Lỗi sentence_idx)"`
- Lỗi khác → trả `"N/A (Lỗi: ...)"`


## 5) Tải file kết quả thô (raw results)
- Mở `raw_results_file` và `json.load(...)`
- Nếu không thấy file:
  - báo lỗi
  - nhắc chạy Bước 2c trước
- Nếu thành công:
  - `all_search_results` là list kết quả, mỗi phần tử tương ứng một query


## 6) Phân tích & tra cứu text cho toàn bộ kết quả

## 6.1 Với mỗi result (mỗi query)
- Lấy:
  - `query_info = result['query_info']`

### (1) Tra cứu text cho QUERY
Gọi:
- `q_text, q_arg_text = get_text_data(query_info['source_dir'], query_info['corpus_id'], query_info['sentence_idx'], query_info['span_id'])`

Tạo object query giàu thông tin:
- `enriched_query` gồm:
  - `corpus_id`, `sentence_idx`, `span_id`
  - `sentence_text` (text câu)
  - `argument_text` (text span)

## 6.2 Tra cứu text cho NEIGHBORS (Top-K)
- Duyệt `for neighbor in result['neighbors']`
- Lấy metadata của match:
  - `match_meta = neighbor['match_metadata']`

Tra cứu:
- `n_text, n_arg_text = get_text_data(match_meta['source_dir'], match_meta['corpus_id'], match_meta['sentence_idx'], match_meta['span_id'])`

Tạo object neighbor đã enrich:
- `total_L2_distance_whitened`: tổng khoảng cách (từ Bước 2c)
- `corpus_id`, `sentence_idx`, `span_id`
- `sentence_text`, `argument_text`
- `original_pt_file`: `match_meta['vector_file_path']` (giữ để debug/trace)

## 6.3 Gộp kết quả cuối cho mỗi query
Mỗi phần tử trong `final_analyzed_results` có dạng:
- `query`: enriched query
- `top_k_neighbors`: danh sách neighbors đã enrich

## 7) Lưu kết quả cuối cùng
- Ghi `final_analyzed_results` ra `final_output_file` bằng `json.dump(..., indent=4)`
- Nếu lỗi khi ghi file → in lỗi.

## Kết quả đầu ra
File:
- `final_analyzed_results_top{K}_v2_WHITENED_L2.json`

Chứa:
- text câu và argument của query
- text câu và argument của từng neighbor
- tổng khoảng cách L2 (đã whiten) để đối chiếu độ tương đồng
- đường dẫn file `.pt` gốc của neighbor để trace lại dữ liệu

In [ ]:
import os
import json
from tqdm.auto import tqdm
import functools

print("--- Bắt đầu Bước 3: Phân tích và Tra cứu Văn bản (v2 - FAISS L2) ---")

# --- 1. CẤU HÌNH ---
K = 5 # Phải khớp với K ở Bước 2c

# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')
output_results_dir = os.path.join(drive_base_path, 'search_results')

# --- File INPUT (Kết quả thô từ Bước 2c) ---
raw_results_file = os.path.join(output_results_dir, f'search_results_top{K}_v2_WHITENED_L2.json')

# --- File OUTPUT (Kết quả cuối cùng đã tra cứu) ---
final_output_file = os.path.join(output_results_dir, f'final_analyzed_results_top{K}_v2_WHITENED_L2.json')

# --- ÁNH XẠ (MAP) ĐẾN CÁC THƯ MỤC DỮ LIỆU GỐC ---
# (Ánh xạ này giống hệt file analyze_search_results_v2.py)
CLEAN_DATA_MAP = {
    # Thư mục Train (dùng cho neighbors)
    'span_adaptation_vectors_train_gramvar_inner_content': os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Train'),
    'span_adaptation_vectors_train_parave_inner_content': os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Train'),

    # Thư mục Test (dùng cho queries)
    'span_adaptation_vectors_test_gramvar_inner_content': os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_GramVar/Test'),
    'span_adaptation_vectors_test_parave_inner_content': os.path.join(drive_base_path, 'Clean_Dataset/Corpus/Split_ParaVE/Test')
}

print(f"File input (kết quả thô): {raw_results_file}")
print(f"File output (phân tích): {final_output_file}")
print(f"Đã ánh xạ {len(CLEAN_DATA_MAP)} thư mục dữ liệu gốc.")

# --- 3. HÀM TRA CỨU (CÓ CACHE) ---

@functools.lru_cache(maxsize=None) # Tăng tốc bằng cách cache lại file json đã đọc
def get_original_data(base_data_path, corpus_id, suffix):
    """ Tải file json gốc của một corpus. """
    json_path = os.path.join(base_data_path, f"{corpus_id}{suffix}")
    try:
        with open(json_path, 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        print(f"Cảnh báo: Không tìm thấy file dữ liệu gốc: {json_path}")
        return None
    except Exception as e:
        print(f"Lỗi khi đọc {json_path}: {e}")
        return None

def get_text_data(source_dir, corpus_id, sentence_idx, span_id):
    """
    Tra cứu văn bản (text) của câu và của span (argument)
    từ file .json gốc.
    """
    base_data_path = CLEAN_DATA_MAP.get(source_dir)
    if not base_data_path:
        return "N/A (Lỗi ánh xạ source_dir)", "N/A"

    # Xác định file _train_set.json hay _test_set.json
    file_suffix = '_test_set.json' if 'test' in source_dir else '_train_set.json'

    # Tải (hoặc lấy từ cache) file data gốc
    original_data = get_original_data(base_data_path, corpus_id, file_suffix)
    if original_data is None:
        return "N/A (Không tìm thấy file data)", "N/A"

    try:
        # Lấy đúng item (câu)
        item = original_data[sentence_idx]
        sentence_text = item.get('text', 'N/A (Không có text)')

        # Lấy đúng argument text
        # Chuyển đổi 'ARG_1' (từ span_id) -> 'ARG-1' (key trong file json)
        arg_key = span_id.replace('_', '-')
        argument_text = item.get('arguments', {}).get(arg_key, 'N/A (Không có arg)')

        return sentence_text, argument_text

    except IndexError:
        return "N/A (Lỗi sentence_idx)", "N/A"
    except Exception as e:
        return f"N/A (Lỗi: {e})", "N/A"

# --- 4. THỰC THI PHÂN TÍCH ---
print("\nĐang tải file kết quả thô...")
try:
    with open(raw_results_file, 'r', encoding='utf-8') as f:
        all_search_results = json.load(f)
    print(f" -> Tải thành công {len(all_search_results)} kết quả truy vấn.")
except FileNotFoundError:
    print(f"LỖI: Không tìm thấy file '{raw_results_file}'.")
    print("Vui lòng chạy file 'run_similarity_search_v2_whitened_FAISS.py' (Bước 2c) trước.")
    exit()
except Exception as e:
    print(f"LỖI: Không thể tải file kết quả thô: {e}")
    exit()

final_analyzed_results = []

print(f"\nBắt đầu tra cứu văn bản cho {len(all_search_results)} truy vấn...")
for result in tqdm(all_search_results, desc="Phân tích & Tra cứu Text"):

    query_info = result['query_info']

    # 1. Tra cứu text cho QUERY (mẫu test)
    q_text, q_arg_text = get_text_data(
        query_info['source_dir'],
        query_info['corpus_id'],
        query_info['sentence_idx'],
        query_info['span_id']
    )

    enriched_query = {
        'corpus_id': query_info['corpus_id'],
        'sentence_idx': query_info['sentence_idx'],
        'span_id': query_info['span_id'],
        'sentence_text': q_text,
        'argument_text': q_arg_text
    }

    # 2. Tra cứu text cho NEIGHBORS (các mẫu train)
    enriched_neighbors = []
    for neighbor in result['neighbors']:
        match_meta = neighbor['match_metadata']

        n_text, n_arg_text = get_text_data(
            match_meta['source_dir'],
            match_meta['corpus_id'],
            match_meta['sentence_idx'],
            match_meta['span_id']
        )

        enriched_neighbors.append({
            'total_L2_distance_whitened': neighbor['total_L2_distance_whitened'],
            'corpus_id': match_meta['corpus_id'],
            'sentence_idx': match_meta['sentence_idx'],
            'span_id': match_meta['span_id'],
            'sentence_text': n_text,
            'argument_text': n_arg_text,
            'original_pt_file': match_meta['vector_file_path'] # Giữ lại để tham khảo
        })

    # 3. Gộp lại
    final_analyzed_results.append({
        'query': enriched_query,
        'top_k_neighbors': enriched_neighbors
    })

print(" -> Tra cứu hoàn tất!")

# --- 5. LƯU KẾT QUẢ CUỐI CÙNG ---
print(f"\nĐang lưu {len(final_analyzed_results)} kết quả đã phân tích...")
try:
    with open(final_output_file, 'w', encoding='utf-8') as f:
        json.dump(final_analyzed_results, f, indent=4)
    print(f" -> ĐÃ LƯU THÀNH CÔNG file phân tích cuối cùng:")
    print(f"    {final_output_file}")
except Exception as e:
    print(f"LỖI khi lưu file kết quả cuối cùng: {e}")

print("\n--- BƯỚC 3 (PHÂN TÍCH) HOÀN TẤT! ---")

--- Bắt đầu Bước 3: Phân tích và Tra cứu Văn bản (v2 - FAISS L2) ---
File input (kết quả thô): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/search_results_top5_v2_WHITENED_L2.json
File output (phân tích): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/final_analyzed_results_top5_v2_WHITENED_L2.json
Đã ánh xạ 4 thư mục dữ liệu gốc.

Đang tải file kết quả thô...
 -> Tải thành công 2410 kết quả truy vấn.

Bắt đầu tra cứu văn bản cho 2410 truy vấn...


Phân tích & Tra cứu Text:   0%|          | 0/2410 [00:00<?, ?it/s]

 -> Tra cứu hoàn tất!

Đang lưu 2410 kết quả đã phân tích...
 -> ĐÃ LƯU THÀNH CÔNG file phân tích cuối cùng:
    /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/final_analyzed_results_top5_v2_WHITENED_L2.json

--- BƯỚC 3 (PHÂN TÍCH) HOÀN TẤT! ---
